In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import time

from numba import jit
import xgbfir
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import os

/home/puneetj/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# importing the files
start = time.time()
train_data = pd.read_csv("/home/saurabhg/PuertoSergo/train.csv")
test_data = pd.read_csv("/home/saurabhg/PuertoSergo/test.csv")
print(time.time()-start)

KeyboardInterrupt: 

In [ ]:
target = train_data["target"]
#del trn_df["target"]

In [ ]:
train_copy = train_data
train_copy = train_copy.replace(-1, np.NaN)
train_copy.isnull().sum()/train_copy.shape[0]*100
# train_copy.isnull().sum()

In [ ]:
train_copy.shape[0]

In [ ]:
print(os.getcwd()+'/')

In [ ]:
# Defining the Normalized Gini method
@jit
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_lgb(act, preds):
#    labels = dtrain.get_label()
    gini_score = gini_normalized(act, preds)
    return 'gini', gini_score,True


In [ ]:
# Defining the Normalized Gini method
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))
 


In [ ]:
a = [.7,.6,.77,.8]
b = [1,1,0,0]
eval_gini(b,a)

In [ ]:
target_train = train_data['target'].values
id_test = test_data['id'].values

df_train=train_data.drop(['target','id'],axis=1)
df_test=test_data.drop(['id'], axis = 1)
combine= pd.concat([df_train,df_test],axis=0)

In [ ]:
# combine.shape
# df_train.shape
# df_test.shape
combine.head()

In [ ]:
# Feature engg code --- will be standard for all the models

combine['ps_ind_06_13_16_18_bin_sum'] = combine.ps_ind_06_bin + combine.ps_ind_07_bin + combine.ps_ind_08_bin + combine.ps_ind_09_bin + combine.ps_ind_10_bin + combine.ps_ind_11_bin + combine.ps_ind_12_bin + combine.ps_ind_13_bin + combine.ps_ind_16_bin + combine.ps_ind_17_bin + combine.ps_ind_18_bin   

combine['ps_reg_01_square'] = combine['ps_reg_01']*combine['ps_reg_01']
#combine['ps_reg_01_cube'] = combine['ps_reg_01']*combine['ps_reg_01']*combine['ps_reg_01']

combine['ps_reg_02_square'] = combine['ps_reg_02']*combine['ps_reg_02']
#combine['ps_reg_02_cube'] = combine['ps_reg_02']*combine['ps_reg_02']*combine['ps_reg_02']

combine['ps_reg_03_square'] = combine['ps_reg_03']*combine['ps_reg_03']
#combine['ps_reg_03_cube'] = combine['ps_reg_03']*combine['ps_reg_03']*combine['ps_reg_03']

combine['ps_car_12_square'] = combine['ps_car_12']*combine['ps_car_12']
#combine['ps_car_12_cube'] = combine['ps_car_12']*combine['ps_car_12']*combine['ps_car_12']

combine['ps_car_13_square'] = combine['ps_car_13']*combine['ps_car_13']
#combine['ps_car_13_cube'] = combine['ps_car_13']*combine['ps_car_13']*combine['ps_car_13']

combine['ps_car_14_square'] = combine['ps_car_14']*combine['ps_car_14']
#combine['ps_car_14_cube'] = combine['ps_car_14']*combine['ps_car_14']*combine['ps_car_14']

combine['ps_car_15_square'] = combine['ps_car_15']*combine['ps_car_15']
#combine['ps_car_15_cube'] = combine['ps_car_15']*combine['ps_car_15']*combine['ps_car_15']

#combine['ps_calc_15_20_bin_sum'] = combine.ps_calc_15_bin + combine.ps_calc_16_bin + combine.ps_calc_17_bin + combine.ps_calc_18_bin + combine.ps_calc_19_bin + combine.ps_calc_20_bin

combine['Interaction_1'] = combine['ps_car_13']*combine['ps_reg_03']
combine['Interaction_2'] = combine['ps_car_13']*combine['ps_ind_05_cat']
combine['Interaction_3'] = combine['ps_car_13']*combine['ps_ind_05_cat']*combine['ps_reg_03']
 

combine['ps_car_11_ct_num'] = combine.ps_car_11_cat
combine['ps_car_11_ct_num_square'] = combine['ps_car_11_ct_num']*combine['ps_car_11_ct_num']

combine = combine.drop(combine[[x for x in combine.columns if "_calc_" in x]] ,axis =1)

for c in combine.columns:
    if '_bin' not in c and '_cat' not in c: #standard arithmetic
        combine[c+str('_median_range')] = (combine[c].values > combine[c]).astype(np.int)
        combine[c+str('_mean_range')] = (combine[c].values > combine[c]).astype(np.int)

In [ ]:
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]
start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'% (name1, n_c + 1, (time.time() - start) / 60))
    print('\r' * 75)
    combine[name1] = combine[f1].apply(lambda x: str(x)) + "_" + combine[f2].apply(lambda x: str(x))
#    sub_df[name1] = sub_df[f1].apply(lambda x: str(x)) + "_" + sub_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(combine[name1].values))
    combine[name1] = lbl.transform(list(combine[name1].values))
#   sub_df[name1] = lbl.transform(list(sub_df[name1].values))

In [ ]:
# Creating the final data sets for training the testing
trn_df=combine[:df_train.shape[0]]
sub_df=combine[df_train.shape[0]:]

In [ ]:
def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
f_cats = [f for f in combine.columns if "_cat" in f]
x
for f in f_cats if "_plus" not in f:
    trn_df[f + "_avg"], sub_df[f + "_avg"] = target_encode(trn_series=trn_df[f],
                                             tst_series=sub_df[f],
                                             target=target,
                                             min_samples_leaf=200,
                                             smoothing=10,
                                             noise_level=0)

In [ ]:
trn_df.head()

In [ ]:
n_splits = 5
n_estimators = 1200
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=15) 
imp_df = np.zeros((len(trn_df.columns), n_splits))
lgb_evals = np.zeros((n_estimators, n_splits))
oof = np.empty(len(trn_df))
sub_preds = np.zeros(len(sub_df))
increase = True
np.random.seed(0)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(target, target)):
    trn_dat, trn_tgt = trn_df.iloc[trn_idx], target.iloc[trn_idx]
    val_dat, val_tgt = trn_df.iloc[val_idx], target.iloc[val_idx]

    clf = lgb.LGBMModel(n_estimators=n_estimators,
                        num_leaves=35,
                        objective="binary",
                        learning_rate=.01, 
                        subsample=.8, 
                        colsample_bytree=.8,
                        min_child_samples= 500,
                    #    is_unbalance = True,
                     #   gamma=1,
                        reg_alpha=0,
                        reg_lambda=1,
                      #  nthread=2
                       )
    # Upsample during cross validation to avoid having the same samples
    # in both train and validation sets
    # Validation set is not up-sampled to monitor overfitting
    if increase:
        # Get positive examples
        pos = pd.Series(trn_tgt == 1)
        # Add positive examples
        trn_dat = pd.concat([trn_dat, trn_dat.loc[pos]], axis=0)
        trn_tgt = pd.concat([trn_tgt, trn_tgt.loc[pos]], axis=0)
        # Shuffle data
        idx = np.arange(len(trn_dat))
        np.random.shuffle(idx)
        trn_dat = trn_dat.iloc[idx]
        trn_tgt = trn_tgt.iloc[idx]
        
    clf.fit(trn_dat, trn_tgt, 
            eval_set=[(trn_dat, trn_tgt), (val_dat, val_tgt)],
            categorical_feature = f_cats,
            eval_metric=gini_lgb,
            early_stopping_rounds=None,
            verbose=False)
    # Find best round for validation setA
    lgb_evals[:, fold_] = clf.evals_result_["valid_1"]["gini"]
    #print(clf.evals_result_ )
    # Keep feature importances
    imp_df[:, fold_] = clf.feature_importances_
    #Xgboost provides best round starting from 0 so it has to be incremented
    best_round = np.argsort(lgb_evals[:, fold_])[::-1][0]

    # Predict OOF and submission probas with the best round
    oof[val_idx] = clf.predict(val_dat, num_iteration=best_round)
    # Update submission
    sub_preds += clf.predict(sub_df, num_iteration=best_round)/ n_splits

    # Display results
    print("Fold %2d : %.6f @%4d / best score is %.6f @%4d"
          % (fold_ + 1,
             gini_normalized(val_tgt, oof[val_idx]),
             n_estimators,
             lgb_evals[best_round, fold_],
             best_round))
          
print("Full OOF score : %.6f" % gini_normalized(target, oof))

In [ ]:
mean_eval = np.mean(lgb_evals, axis=1)
std_eval = np.std(lgb_evals, axis=1)
best_round = np.argsort(lgb_evals)[::-1][0]
#print("Best mean score : %.6f + %.6f @%4d"% (mean_eval[best_round], std_eval[best_round], best_round))

In [ ]:
importances = sorted([(trn_df.columns[i], imp) for i, imp in enumerate(imp_df.mean(axis=1))],
                     key=lambda x: x[1])

for f, imp in importances[::-1]:
    print("%-34s : %10.4f" % (f, imp))
    
sub_df["target"] = sub_preds

sub_df[["target"]].to_csv("submission_10262017_lgb_5s_first.csv", index=False, float_format="%.9f")

In [ ]:
sub = pd.DataFrame()
sub['id'] = test_data["id"]
sub["target"] = sub_preds
sub.to_csv("submission_10302017_lgb_5s_with_0.01_removing_cube_10Fold.csv", index=False, float_format="%.9f")

In [ ]:
sub_df.head()